In [1]:
import sys
import pandas as pd
import copy
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import itertools
sys.path.append("C:/dev/pyktrader3/")
from pycmqlib3.utility import misc, dbaccess
from bktest import backtest
import pycmqlib3.analytics.data_handler as dh


In [2]:
def load_daily_by_product(cnx, product_code, exch, d_start, d_end):
    price_fields = ['instID', 'exch', 'date', 'open', 'high', 'low', 'close', 'volume', 'openInterest']
    stmt = "select {variables} from fut_daily where instID like '{prod_code}%' and exch = '{exch}' ".format( \
                                    variables=','.join(price_fields),
                                    prod_code = product_code, exch = exch)
    stmt = stmt + "and date >= '%s' " % d_start.strftime('%Y-%m-%d')
    stmt = stmt + "and date <= '%s' " % d_end.strftime('%Y-%m-%d')
    df = pd.io.sql.read_sql(stmt, cnx)
    return df

def process_index_rank(df, func, ma_win, roll_win = '30b'):
    df['calc_field'] = df.apply(func, axis = 1)
    df['expiry'] = df['instID'].apply(lambda x: misc.contract_expiry)
    ptab = pd.pivot_table(df, values = 'calc_field', index=['date'], columns=['instID'], aggfunc='mean', fill_value=None,)
    ptab['rank_field'] = df['calc_field'].rolling(ma_win).mean()
    

In [15]:
cnx = dbaccess.connect(**dbaccess.dbconfig)
adf = load_daily_by_product(cnx, 'rb', 'SHFE', datetime.date(2017,1,1), datetime.date(2021,2,5))
print(adf.tail())


       instID  exch        date    open    high     low   close  volume  \
11784  rb2201  SHFE  2021-02-01  4098.0  4121.0  4059.0  4061.0    5274   
11785  rb2201  SHFE  2021-02-02  4080.0  4080.0  3991.0  4002.0    9679   
11786  rb2201  SHFE  2021-02-03  4005.0  4042.0  3982.0  4022.0    7365   
11787  rb2201  SHFE  2021-02-04  4035.0  4070.0  4017.0  4064.0    5063   
11788  rb2201  SHFE  2021-02-05  4067.0  4097.0  4050.0  4085.0    5742   

       openInterest  
11784         19276  
11785         23691  
11786         24056  
11787         23814  
11788         23477  


In [16]:
ts = pd.Series(True, index=adf.index)
#for cont_mth in [1, 5, 10]:
#    ts = ts | adf['instID'].str.endswith("%02d" % (cont_mth,), na=False)
adf = adf[ts]
print(adf)
adf['calc_field'] = adf.apply(lambda x: x['volume']*0.5 + x['openInterest']*0.5, axis = 1)
pdf = pd.pivot_table(adf, values='calc_field', index='date', columns=['instID']).rolling(5).mean()
print(pdf.columns)
rank_df = pdf.rank(axis=1, ascending=False)
rank_df2 = pdf.apply(np.argsort, axis=1)
print(rank_df, rank_df2)

       instID  exch        date    open    high     low   close  volume  \
0      rb1701  SHFE  2017-01-03  2988.0  2999.0  2925.0  2941.0    3302   
1      rb1701  SHFE  2017-01-04  2935.0  2990.0  2908.0  2980.0    3420   
2      rb1701  SHFE  2017-01-05  2955.0  3008.0  2953.0  3008.0    2520   
3      rb1701  SHFE  2017-01-06  3013.0  3013.0  2960.0  2960.0     600   
4      rb1701  SHFE  2017-01-09  3008.0  3039.0  2981.0  3039.0    2040   
...       ...   ...         ...     ...     ...     ...     ...     ...   
11784  rb2201  SHFE  2021-02-01  4098.0  4121.0  4059.0  4061.0    5274   
11785  rb2201  SHFE  2021-02-02  4080.0  4080.0  3991.0  4002.0    9679   
11786  rb2201  SHFE  2021-02-03  4005.0  4042.0  3982.0  4022.0    7365   
11787  rb2201  SHFE  2021-02-04  4035.0  4070.0  4017.0  4064.0    5063   
11788  rb2201  SHFE  2021-02-05  4067.0  4097.0  4050.0  4085.0    5742   

       openInterest  
0             12360  
1             10320  
2              8520  
3          